## Initialization Cells

In [ ]:
# Standard includes
%matplotlib inline
# %matplotlib widget
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot
# from scipy.optimize import curve_fit
# from scipy.stats import norm
# from statsmodels.base.model import GenericLikelihoodModel

In [ ]:
# Style setup
import seaborn as sns
sns.set_palette('muted')
sns.set_color_codes()
sns.set_style('ticks')
sns.set_style({'xtick.direction': 'in','ytick.direction': 'in'})
sns.set_style({'axes.grid': 'True', 'grid.color': '0.95'})

plt.rcParams['figure.figsize'] = [6,6]
plt.rcParams['figure.dpi'] = 100

def darken_color(color, p):
    return (color[0]*p,color[1]*p,color[2]*p)

import matplotlib as mpl
colors = sns.color_palette('muted') + [(.1, .1, .1)]
for code, color in zip(['bd','gd','rd','md','yd','cd','kd'], colors):
    rgb = mpl.colors.colorConverter.to_rgb(darken_color(color,0.8))
    mpl.colors.colorConverter.colors[code] = rgb
    mpl.colors.colorConverter.cache[code] = rgb

blue = (114/256, 147/256, 203/256)
orange = (225/256, 151/256,  76/256)
green = (132/256, 186/256,  91/256)
red = (211/256,  94/256,  96/256)
grey = (128/256, 133/256, 133/256)
violet = (144/256, 103/256, 167/256)
brown = (171/256, 104/256,  87/256)
yellow = (204/256, 194/256,  16/256)

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def apply_cuts(df):
    """Apply common cuts to a dataset and return the result."""
    return df[ 
        (df.vrusable == 1) &
        (df.vtusable == 1) &
        (((df.vrchi2/df.vrndf) < 50) | (df.vrntrk == 1)) &
        (((df.vtchi2/df.vtndf) < 50) | (df.vtntrk == 1)) &
        (((np.sqrt(df.vrerr6) < 0.02) & (df.vrntrk > 1)) | ((np.sqrt(df.vrerr6) < 0.05) & (df.vrntrk == 1))) &
        (((np.sqrt(df.vterr6) < 0.02) & (df.vtntrk > 1)) | ((np.sqrt(df.vterr6) < 0.05) & (df.vtntrk == 1))) &
        (df.csbdtg > -0.6) &
        ((df.de > -0.14) & (df.de < 0.068)) &
        ((df.dt > -10) & (df.dt < 10)) &
        (df.thetab > 0.65) & (df.thetab < 2.95) &
        (df.nocand <= 3)
    ]

def get_dataframe(paths, branches=None):
    """Read ROOT files, apply common cuts, and return as dataframe."""
    assert paths, "No paths supplied"
    df = pd.DataFrame()
    for path in paths:
        temp_df = uproot.open(path)["h2000"].pandas.df()
        temp_df = apply_cuts(temp_df)
        if branches:
            if isinstance(branches, str):
                branches = [branches]
            df = df.append(temp_df[branches])
        else:
            df = df.append(temp_df)
    return df

## Exploration

In [ ]:
df_mc   = get_dataframe(glob.glob('../data/Kpi/realistic_mc/stream0/*.root'))
df_data = get_dataframe(glob.glob('../data/Kpi/*.root'))

In [ ]:
df_data = df_data.head(df_mc.shape[0])
print(df_mc.shape)
print(df_data.shape)

In [ ]:
fig, ax = plt.subplots()
df_mc.plot.scatter(x='mdspi', y='mdspi0', c='evmcflag', s=1, figsize=(6,5), ax=ax, cmap="viridis")
plt.xlabel("M($D^{*}\pi$)")
plt.ylabel("M($D^{*}\pi^0$)")
fig.get_axes()[1].set_ylabel("MC Flag")

plt.tight_layout()
plt.savefig("mdstpi_vs_mdstpi0.png")
plt.savefig("mdstpi_vs_mdstpi0.pdf")

In [ ]:
vals, bins, patches = plt.hist([df_mc.mdspi , df_data.mdspi], histtype='step', bins=30, linewidth=1.5)

# Add error bars
centers = (bins[:-1] + bins[1:]) / 2
width =  bins[1] - bins[0]
plt.gca().set_prop_cycle(None)
for val in vals:
    plt.errorbar(centers, val, yerr=np.sqrt(val), drawstyle="steps-mid", ls="none")

plt.legend(["MC", "data"], loc="upper left")
plt.xlabel("M($D^{*}\pi$) [GeV]")
plt.savefig("dspimass.pdf")
plt.savefig("dspimass.png")

In [ ]:
vals, bins, patches = plt.hist([df_mc.mdspi0 , df_data.mdspi0], histtype='step', bins=30, linewidth=1.5)

# Add error bars
centers = (bins[:-1] + bins[1:]) / 2
width =  bins[1] - bins[0]
plt.gca().set_prop_cycle(None)
for val in vals:
    plt.errorbar(centers, val, yerr=np.sqrt(val), drawstyle="steps-mid", ls="none")

plt.legend(["MC", "data"], loc="upper left")
plt.xlabel("M($D^{*}\pi^0$) [GeV]")
plt.savefig("dspi0mass.pdf")
plt.savefig("dspi0mass.png")

In [ ]:
plt.hist([df_mc.mdspi0, df_data.mdspi0], histtype='step', bins=30, density=True, linewidth=1.5)
plt.legend(["MC", "data"], loc="upper left")

In [ ]:
vals, bins, patches = plt.hist([df_mc.rhomass , df_data.rhomass], histtype='step', bins=30, linewidth=1.5)

# Add error bars
centers = (bins[:-1] + bins[1:]) / 2
width =  bins[1] - bins[0]
plt.gca().set_prop_cycle(None)
for val in vals:
    plt.errorbar(centers, val, yerr=np.sqrt(val), drawstyle="steps-mid", ls="none")

plt.legend(["MC", "data"], loc="upper left")
plt.xlabel("M($\pi\pi^0$) [GeV]")
plt.savefig("rhomass.pdf")
plt.savefig("rhomass.png")